In [2]:
import pandas as pd
import numpy as np

Paths

In [3]:
generated_data = 'generated_data/'
quotebank_2019_path = generated_data + "quotes-2019-nytimes.json.bz2"
qids_onlyquotebank_path = generated_data + "wikidata_labels_descriptions_quotebank.csv.bz2"
qids_wikidata_path  = generated_data + "wikidata_labels_descriptions.csv.bz2"

## Load the quotebank sample

In [4]:
quotebank_2019_sample = pd.read_json('generated_data/quotes-2019-nytimes.json.bz2', lines=True, compression='bz2')
quotebank_2019_sample.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E
1,2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E
2,2019-05-09-055187,It makes it much more difficult for him to mak...,None,[],2019-05-09 18:11:29,1,"[[None, 0.6493], [President Bill Clinton, 0.27...",[http://mobile.nytimes.com/2019/05/09/world/as...,E
3,2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E
4,2019-01-04-001792,A Pile of Leaves.,None,[],2019-01-04 10:00:07,1,"[[None, 0.8737], [Jason Fulford, 0.1263]]",[https://www.nytimes.com/2019/01/04/books/revi...,E


extract the QID colum that we want to attribute labels / description to

In [5]:
qids = quotebank_2019_sample['qids']
qids.head()

0    [Q16213953, Q20707104, Q43143598, Q58886302, Q...
1                                            [Q105756]
2                                                   []
3                                           [Q1855840]
4                                                   []
Name: qids, dtype: object

In [6]:
probas = quotebank_2019_sample['probas']
probas.head()

0             [[James Fisher, 0.7475], [None, 0.2525]]
1              [[John Updike, 0.5856], [None, 0.4144]]
2    [[None, 0.6493], [President Bill Clinton, 0.27...
3    [[Xavier Becerra, 0.9065], [None, 0.0909], [St...
4            [[None, 0.8737], [Jason Fulford, 0.1263]]
Name: probas, dtype: object

Seems like some lines have multiple labels assigned to them, this is probably because there are many people with the same names in wikidata. 

In [7]:
print(qids[0])
print(probas[0])

['Q16213953', 'Q20707104', 'Q43143598', 'Q58886302', 'Q6133913']
[['James Fisher', '0.7475'], ['None', '0.2525']]


TO DO : find a better way to choose QID
For now, we take only the first qid because it will be more practical

In [8]:
def first_qid(x):
    if x:
        x = x[0]
    else:
        x = '0'
    return x

def clean_qids(df):
    return df.apply(first_qid)

qids_clean = clean_qids(qids)

qids_clean.head()

0    Q16213953
1      Q105756
2            0
3     Q1855840
4            0
Name: qids, dtype: object

## Loading the QIDs that have attributes that exist in the parquet file

In [9]:
qids_onlyquotebank = pd.read_csv(qids_onlyquotebank_path, compression='bz2', index_col='QID')
qids_onlyquotebank.head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America


Seems like we're missing some QIDs ( the QIDs in the parquet file don't contain all the QIDs of quotebank), so this will not be enough

In [10]:
(qids_onlyquotebank['Label'].isin(['James Fisher'])).any()

False

In [11]:
(qids_onlyquotebank.index=='Q16213953').any()

False

In [12]:
(qids_onlyquotebank.index=='Q31').any()

True

If the QID and it's label is present qids_onlyquotebank, assign the label to the QID in the 2019 sample :

In [13]:
qids_clean = qids_clean.to_frame()

In [14]:
qids_clean.head()

,qids
0,Q16213953
1,Q105756
2,0
3,Q1855840
4,0


In [15]:
type(qids_clean)

pandas.core.frame.DataFrame

In [16]:
qids_clean['qids'].iloc[1]

'Q105756'

In [17]:
qids_onlyquotebank.head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America


In [18]:
(qids_onlyquotebank.index == 'Q9682').any()

True

In [19]:
d='Q9682'
qids_onlyquotebank.index.isin([d]).any()

True

In [20]:
qids_onlyquotebank.loc['Q9682']

Label                                               Elizabeth II
Description    queen of the UK, Canada, Australia, and New Ze...
Name: Q9682, dtype: object

In [21]:
qids_clean.head()

,qids
0,Q16213953
1,Q105756
2,0
3,Q1855840
4,0


In [22]:
qids_clean.where(qids_clean['qids']=='Q9682').dropna().head()

,qids
4149,Q9682
15969,Q9682
24134,Q9682
29023,Q9682
36520,Q9682


In [23]:
qids_clean_merged = pd.merge(qids_clean, qids_onlyquotebank, right_index=True, left_on='qids',how='left')

replacing the NaN values with an empty string

In [26]:
#qids_clean_merged.fillna(value='',inplace=True)

In [27]:
qids_clean_merged.head()

,qids,Label,Description
0,Q16213953,NaN,NaN
1,Q105756,NaN,NaN
2,0,NaN,NaN
3,Q1855840,NaN,NaN
4,0,NaN,NaN


Checking that the Label have indeed been merged

In [28]:
qids_clean_merged['Label'].iloc[4149]

'Elizabeth II'

A lot of values are equal to nan so we will try to look for more qids in the entire wikidata qid dataset given by the TAS

# Load all the qids from wikidata

In [39]:
def process_chunk(chunk, qids_clean_merged):
    """
    this function processes one chunk of data
    """
    qids_clean_merged = qids_clean_merged.reindex(columns=qids_clean_merged.columns.union(chunk.columns))
    
    qids_clean_merged.update(chunk)

    return qids_clean_merged

def process_qid_one(path_to_file, qids_clean, chunksize = 10 ** 4):
    """
    this function processes a csv file in chunks
    """
    qids_clean_merged = qids_clean.copy()
    qids_clean_merged['Label']=np.NaN
    qids_clean_merged['Description']=np.NaN
    qids_clean_merged.set_index('qids',inplace=True)
    qids_clean_merged.index.rename('QID', inplace=True)
    
    with pd.read_csv(path_to_file, compression='bz2', index_col='QID', chunksize=chunksize) as df_reader:
        for chunk in df_reader:
            qids_clean_merged = process_chunk(chunk, qids_clean_merged)
    return qids_clean_merged
'''def process_qid_all(Quotebank_tuple, qids_clean, chunksize = 10 ** 4):
    """
    this function processes multiple csv files using,
    another function that process them in chunks
    """
    years = ('2020','2019','2018','2017','2016','2015')

    for i in range(len(Quotebank_tuple)):
        print('__________________ year : ', years[i],' ________________')
        qids_clean_merged = process_qid_one(Quotebank_tuple[i], qids_clean, chunksize = chunksize)
    return qids_clean_merged'''

'def process_qid_all(Quotebank_tuple, qids_clean, chunksize = 10 ** 4):\n    """\n    this function processes multiple csv files using,\n    another function that process them in chunks\n    """\n    years = (\'2020\',\'2019\',\'2018\',\'2017\',\'2016\',\'2015\')\n\n    for i in range(len(Quotebank_tuple)):\n        print(\'__________________ year : \', years[i],\' ________________\')\n        qids_clean_merged = process_qid_one(Quotebank_tuple[i], qids_clean, chunksize = chunksize)\n    return qids_clean_merged'

In [41]:
pd.read_csv(qids_wikidata_path, compression='bz2', index_col='QID',nrows=100).head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q8,happiness,mental or emotional state of well-being charac...
Q23,George Washington,1st president of the United States (1732–1799)
Q24,Jack Bauer,character from the television series 24
Q42,Douglas Adams,English writer and humorist


In [42]:
test2 = process_qid_one(qids_onlyquotebank_path, qids_clean, chunksize = 10 ** 3)

In [43]:
test2.head()

,Label,Description
QID,,
Q16213953,NaN,NaN
Q105756,NaN,NaN
0,NaN,NaN
Q1855840,NaN,NaN
0,NaN,NaN


In [44]:
test2['Label'].iloc[4149]

'Elizabeth II'

In [45]:
test2.loc['Q9682'].head()

,Label,Description
QID,,
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."


In [50]:
qids_clean.head(30)

,qids
0,Q16213953
1,Q105756
2,0
3,Q1855840
4,0
5,Q7817334
6,0
7,0
8,Q3074013
9,0


In [46]:
test2 = process_qid_one(qids_wikidata_path, qids_clean, chunksize = 10 ** 6)

In [47]:
test2.head(30)

,Label,Description
QID,,
Q16213953,James Fisher,British actor
Q105756,John Updike,"American novelist, poet, short story writer, a..."
0,NaN,NaN
Q1855840,Xavier Becerra,Attorney General of California
0,NaN,NaN
Q7817334,Tom Rath,American author
0,NaN,NaN
0,NaN,NaN
Q3074013,Florence Parly,French politician


In [48]:
test2.iloc[4149]

Label                                               Elizabeth II
Description    queen of the UK, Canada, Australia, and New Ze...
Name: Q9682, dtype: object

In [49]:
test2.loc['Q9682']

,Label,Description
QID,,
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
Q9682,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze..."
